In [6]:
import boto3
from zipfile import ZipFile

In [2]:
# Create S3 bucket
s3c = boto3.client('s3', region_name = 'us-east-1')
bucket = s3c.create_bucket(Bucket = 'wos-xml')


In [9]:
# Test extractcall()
with ZipFile('xml/Schema_2023.zip', 'r') as f:
    f.extractall('xml/Schema_2023')

In [3]:
# For now: Read XML from local system
# TODO: Use Lambda func to scrape XML from lib website and write to S3
xml = open('xml/2023_CORE/WR_2023_20240112181857_CORE_0001.xml.gz', 'rb')


In [4]:
# For now: Write zipped XML to S3 bucket
s3r = boto3.resource('s3')
s3r.Bucket('wos-xml').put_object(
                Key = 'xml/2023_CORE/WR_2023_20240112181857_CORE_0001.xml.gz', # File name
                Body = xml) # XML file

s3.Object(bucket_name='wos-xml', key='xml/2023_CORE/WR_2023_20240112181857_CORE_0001.xml.gz')